In [16]:
import tweepy
import sys
import csv
import re
from textblob import TextBlob

In [17]:
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)


In [18]:
consumer_key = 'CONSUMER_KEY'
consumer_secret = 'CONSUMER_SECRET'
access_token = 'ACCESS_TOKEN'
access_secret = 'ACCESS_SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth,wait_on_rate_limit=True)

In [29]:
searchQuery = 'Trump'
maxTweets = 1000
acceptedCharsRegex = '[^0-9a-zA-Z.:/()&@-_=+;?!*\']+'
# header for csv file
headers = [
    'tweet', 
    'created_at',
    'word_count',
    'favorite_count', 
    'retweet_count',
    'user_followers_count',
    'user_following_count',
    'user_friends_count',
    'user_verified',
    'subjectivity', 
    'polarity', 
    'sentiment'
]

In [30]:
threshold = {'neutral_min': -0.02, 'neutral_max': 0.25}

# function to get the sentiment
def get_sentiment(text, analysis):
    if(analysis.sentiment.polarity < threshold['neutral_min']):
        return "Negative"
    elif(analysis.sentiment.polarity >= threshold['neutral_max']):
        return "Positive"
    else:
        return "Neutral"

In [31]:
with open('trump_twitter.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(headers)
    for tweet in tweepy.Cursor(api.search,q=searchQuery, tweet_mode='extended').items(maxTweets):
        # getting the tweet dict
        # print(tweet.__dict__)
        
        # cleaning the tweet text of special chars
        tweetText = re.sub(acceptedCharsRegex, ' ', tweet.full_text.translate(non_bmp_map))
        
        # creating the TextBlobObject for analysis
        analysis = TextBlob(tweetText)
        
        # saving all the information into an array
        tweetInfos = [
            tweetText, 
            tweet.created_at.strftime('%Y-%m-%d'),
            analysis.words.__sizeof__(), 
            tweet.favorite_count,
            tweet.retweet_count,
            tweet.user.followers_count,
            tweet.user.following,
            tweet.user.friends_count,
            tweet.user.verified,
            analysis.sentiment.subjectivity, 
            analysis.sentiment.polarity,
            get_sentiment(tweetText,analysis)
        ]
        
        # print(tweetInfos)
        
        # writing the array in the csv file
        writer.writerow(tweetInfos)
